Dit tijdelijke document verkent de mogelijke targetvariabelen om de gestelde vragen vanuit ProRail te beantwoorden.

ProRail is opzoek naar voorspelling wat betreft functieherstel aan het begin van een storing, door de gegeven dataset van storingen in het verleden.

De geleverde informatie over mogelijke featurevariabelen die volgens de business-expert hoogstwaarschijnlijk grote invloed op de uiteindelijke duur van functieherstel van het spoor zijn, is hoogstwaarschijnlijk pas bij het arriveren van de monteur beschikbaar. Er vanuit gaande dat er dan pas een voorspelling gemaakt kan worden (in ieder geval met het eerste model), moet verkend worden of te tijd van monteur-aankomst tot functieherstel beschikbaar is in de dataset.

Dit document is in afwachting van de conclusie over targetvariabelen vanuit de business-interviews. Om productief te zijn als team, is parralel werken op het moment redelijk essentieel. Omdat ik hier nog op wacht, rust dit document wel op een aantal aannames over de informatie die daar uit zal komen.

Eerst laden we wat libraries, en de dataset in:

In [23]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("sap_storing_data_hu_project.csv")

C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,19,21,24,25,27,32,33,36,37,38,39,40,46,47,48,49,51,53,55,56,60,73,75,78,80,82,83,86,87,91,95,99,102,103,104,107,108,109,110,111,112,113,116,118,120,121,122,123,124,129) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Van de business-interviews weten we dat het te-voorspellen variabele het tijdstip van 'herstel' is. In de dataset staat een kolom voor tijdstip van 'functieherstel' en een tijdstip van 'einde storing'. Laten we deze in makkelijkere variabelenamen opslaan, en inspecteren:

In [24]:
fh = "stm_fh_ddt"
dataset[fh].head(5)

0                    NaN
1    02/01/2006 09:00:00
2    02/01/2006 13:26:00
3    02/01/2006 17:20:00
4    02/01/2006 22:36:00
Name: stm_fh_ddt, dtype: object

In [25]:
es = "stm_sap_storeind_ddt"
dataset[es].head(5)

0                    NaN
1    02/01/2006 09:00:00
2    02/01/2006 13:26:00
3    02/01/2006 17:20:00
4    02/01/2006 22:36:00
Name: stm_sap_storeind_ddt, dtype: object

In eerste oogopslag lijken de waardes in deze tabel gelijk te lijken. Laten we kijken voor welk deel van de dataset dat geld.

In [26]:
def percentage(prob: float) -> float:
    """Convert a probability (p > 0 ∧ p < 1) to a percentage.
    Convenience function.

    Args:
        prob: probability between 0 and 1.

    Returns:
        percentage."""
    return prob * 100


n_rows = dataset.shape[0]
n_equal = (dataset[fh] == dataset[es]).sum()

f"{percentage(n_equal / n_rows)}%"

'19.96210953749065%'

We krijgen hier 16% uit, wat niet enorm overeen lijkt te komen met de resultaten die we handmatig kunnen observeren. Hoe zit het met de datatypes?

In [27]:
dataset[[fh, es]].dtypes

stm_fh_ddt              object
stm_sap_storeind_ddt    object
dtype: object

Objects, laten we dat aanpassen.

In [28]:
dataset[[fh, es]] = dataset[[fh, es]].convert_dtypes()
dataset[[fh, es]].dtypes

stm_fh_ddt              string
stm_sap_storeind_ddt    string
dtype: object

Nu zijn strings. Eigenlijk willen we datetimes, dus laten we kijken of dat aangepast kan. Waarschijnlijk worden er een aantal dan niet meegenomen, dus even controleren hoeveel nans er zijn.

In [29]:
f"{dataset[fh].isna().sum()} and {dataset[es].isna().sum()} out of {dataset.shape[0]}"

'142213 and 160938 out of 463969'

Even kijken of pandas het een beetje automatisch can converten

In [34]:
new_fh = pd.to_datetime(dataset[fh], errors='coerce')
new_se = pd.to_datetime(dataset[es], errors='coerce')

f"{new_fh.isna().sum()} and {new_se.isna().sum()} out of {dataset.shape[0]}"

'142213 and 160938 out of 463969'

Goed genoeg.
Nu we deze omgezet hebben, kunnen we kijken of deze gelijk zijn aan elkaar.

In [40]:
n_equal = (new_fh == new_se).sum()

f"{percentage(n_equal / n_rows)}%"

'19.96210953749065%'

Nog steeds niet heel erg.

Waarschijnlijk willen we van een van deze 2 aftrekken wanneer de monteur aankwam.

Dit is waarschijnlijk opgemaakt uit: stm_aanntpl_tijd en stm_aanntpl_dd.

Of mss stm_aanntpl_ddt, daar staat wel een vraagteken bij in de dataset.

In [42]:
dataset[["stm_aanntpl_tijd", "stm_aanntpl_dd", "stm_aanntpl_ddt"]]

,stm_aanntpl_tijd,stm_aanntpl_dd,stm_aanntpl_ddt
0,NaN,NaN,NaN
1,00:00:00,NaN,NaN
2,00:00:00,NaN,NaN
3,00:00:00,NaN,NaN
4,00:00:00,NaN,NaN
...,...,...,...
463964,10:28:00,10/02/2012,10/02/2012 10:28:00
463965,11:15:00,10/02/2012,10/02/2012 11:15:00
463966,11:41:00,10/02/2012,10/02/2012 11:41:00
463967,11:13:00,10/02/2012,10/02/2012 11:13:00


In [46]:
for col in dataset[["stm_aanntpl_tijd", "stm_aanntpl_dd", "stm_aanntpl_ddt"]].columns:
    print(dataset[col].isna().sum())

4
166521
166634


 Er is een kolom, stm_progfh_in_duur, en een kolom stm_fh_duur. Is een van deze 2 gelijk aan wat we net uitgerekend hebben?

In [35]:
dataset[["stm_progfh_in_duur", "stm_fh_duur"]].head(5)

,stm_progfh_in_duur,stm_fh_duur
0,NaN,NaN
1,99999999.0,0.0
2,99999999.0,51.0
3,99999999.0,40.0
4,99999999.0,6.0


Die eerste is waarschijnlijk niet heel nuttig. Die 2e dan? Dat zal dan waarschijnlijk in minuten, of uren zijn. Allebei bekijken.

In [39]:
tgt1 = "stm_fh_duur"
dataset[tgt1].dtypes

dtype('float64')

In [47]:
dataset[tgt1].isna().sum()

4

In [49]:
dataset[tgt1].head(5)

0     NaN
1     0.0
2    51.0
3    40.0
4     6.0
Name: stm_fh_duur, dtype: float64

Wss is dat het targetvariabele. 